In [91]:
%reload_ext autoreload
%autoreload 2

import musx
import rtmidi
from custom_utils import *

In [2]:
midiout = rtmidi.MidiOut()
outports = midiout.get_ports()
print("available ports:", outports)

available ports: ['Microsoft GS Wavetable Synth 0', 'musx2reaper 1']


In [3]:
midiout.open_port(outports.index('musx2reaper 1'))

In [40]:
def beads(queue):
    queue.compose([
       [0, distribution(queue, chan=0, length=40, notes=musx.scale(0, 7, 2, 2, 1, 2, 2, 2, 1), dur=1)],
       [8, distribution(queue, chan=0, length=32, notes=musx.scale(24, 7, 2, 2, 1, 2, 2, 2, 1), dur=0.5)],
       [16, distribution(queue, chan=0, length=24, notes=musx.scale(36, 7, 2, 2, 1, 2, 2, 2, 1), dur=0.25)],
       [24, distribution(queue, chan=0, length=16, notes=musx.scale(48, 7, 2, 2, 1, 2, 2, 2, 1), dur=0.25)],
       [32, distribution(queue, chan=0, length=8, notes=musx.scale(60, 7, 2, 2, 1, 2, 2, 2, 1), dur=0.125)],
       [40, distribution(queue, chan=0, length=28, notes=musx.scale(36, 7, 2, 2, 1, 2, 2, 2, 1), dur=0.25)],
       [40, cc_linear(queue, chan=0, ctrl=32, length=16, start=0, end=32, low=0, high=127)],
       [56, distribution(queue, chan=0, length=16, notes=musx.scale(0, 7, 2, 2, 1, 2, 2, 2, 1), dur=0.5)],
       [60, distribution(queue, chan=0, length=28, notes=musx.scale(24, 7, 2, 2, 1, 2, 2, 2, 1), dur=0.25)],
       [64, distribution(queue, chan=0, length=24, notes=musx.scale(36, 7, 2, 2, 1, 2, 2, 2, 1), dur=0.125)],
       [68, distribution(queue, chan=0, length=20, notes=musx.scale(48, 7, 2, 2, 1, 2, 2, 2, 1), dur=0.125)],
    ])
    yield 0

In [6]:
def bead_to_blow(queue, *, chan1, chan2, length, notes, rhy, distribution=musx.uniran, lo=0, hi=1):
    start_time = queue.now
    while queue.now - start_time < length:
        selected = round(musx.rescale(distribution(), lo, hi, 0, len(notes) - 1))
        queue.out.addevent(musx.MidiNote(time=queue.now, dur=rhy, key=notes[selected], chan=chan1))
        queue.out.addevent(musx.MidiNote(time=queue.now, dur=rhy, key=notes[selected] + 60, chan=chan2))
        yield rhy

In [7]:
def transition1(queue):
    queue.compose([[0, bead_to_blow(queue, chan1=0, chan2=2, length=16, notes=musx.scale(24, 7, 2, 2, 1, 2, 2, 2, 1), rhy=0.5)],
                   [4, cc_linear(queue, chan=2, ctrl=20, length=12, start=0, end=24, low=0, high=127)],])
    yield 0 

In [10]:
seq = musx.MidiSeq()
queue = musx.Scheduler(out=seq)
# queue.compose([[0, cc_init(queue)],
#                [0, beads(queue)]])
queue.compose([
    [0, cc_init(queue)],
    [0, beads(queue)],
            #    [88, transition1(queue)],
    [88, blown_bottles(queue)],
    [100, cc_linear(queue, chan=2, ctrl=32, length=60, start=0, end=1)],
    [130, swell(queue)],
])         
seq.play(midiout, False)

Done!


In [88]:
midiout = midi_restart(midiout, 'musx2reaper 1')

In [84]:
midiout.send_message(musx.control_change(0, 33, 48)) # used for MIDI learning the Supermassive parameter
# midiout.send_message(musx.control_change(0, 33, 64)) # used for MIDI learning bottle-high pan parameter
# midiout.send_message(musx.control_change(0, 34, 64)) # used for MIDI learning bottle-high pan parameter

In [ ]:
"""
Notes for myself:
- Multitrack is the best way to make sonic material (can pan sounds differently)
- "Chaos theory"
- Super high pitched bottles (108+) are digital like
"""

In [11]:
def speedup(queue):

    queue.compose([
        [0, cc_init(queue)],
        [0, distribution(queue, chan=15, length=8, notes=musx.scale(60, 7, 2, 2, 1, 2, 2, 2, 1), dur=0.5)],
        [8, distribution(queue, chan=15, length=8, notes=musx.scale(60, 7, 2, 2, 1, 2, 2, 2, 1), dur=0.25)],
        [8, distribution(queue, chan=15, length=8, notes=musx.scale(72, 7, 2, 2, 1, 2, 2, 2, 1), dur=0.25)],
        [16, distribution(queue, chan=15, length=8, notes=musx.scale(48, 7, 2, 2, 1, 2, 2, 2, 1), dur=0.125)],
        [16, distribution(queue, chan=15, length=8, notes=musx.scale(60, 7, 2, 2, 1, 2, 2, 2, 1), dur=0.125)],
        [16, distribution(queue, chan=15, length=8, notes=musx.scale(72, 7, 2, 2, 1, 2, 2, 2, 1), dur=0.125)],
        [16, distribution(queue, chan=15, length=8, notes=musx.scale(84, 7, 2, 2, 1, 2, 2, 2, 1), dur=0.125)],
    ])
    seq.play(midiout, False)

In [78]:
def cc_init(queue):
    queue.out.addevent(musx.MidiEvent.control_change(0, 32, 0)) # Ukulele Supermassive mix -> 0%
    queue.out.addevent(musx.MidiEvent.control_change(0, 33, 64)) # Ukulele volume -> 0 dB
    queue.out.addevent(musx.MidiEvent.control_change(2, 32, 0)) # Blown bottle Supermassive mix -> 0%
    queue.out.addevent(musx.MidiEvent.control_change(2, 33, 0)) # Blown bottle volume -> -60 dB
    queue.out.addevent(musx.MidiEvent.control_change(4, 32, 0)) # Swell bottles Supermassive mix -> 0%
    queue.out.addevent(musx.MidiEvent.control_change(4, 33, 64)) # Reset swell bottle 1 panning to center
    queue.out.addevent(musx.MidiEvent.control_change(4, 34, 64)) # Reset swell bottle 2 panning to center
    queue.out.addevent(musx.MidiEvent.control_change(4, 35, 31)) # Swell bottles volume -> -30 dB
    yield 0

In [85]:
def ukulele(queue):
    queue.compose([
        [0, distribution(queue, chan=0, length=40, notes=musx.scale(24, 7, 2, 2, 1, 2, 2, 2, 1), dur=1)],
        [8, distribution(queue, chan=0, length=32, notes=musx.scale(36, 7, 2, 2, 1, 2, 2, 2, 1), dur=0.5)],
        [16, distribution(queue, chan=0, length=24, notes=musx.scale(48, 7, 2, 2, 1, 2, 2, 2, 1), dur=0.25)],
        [24, distribution(queue, chan=0, length=16, notes=musx.scale(60, 7, 2, 2, 1, 2, 2, 2, 1), dur=0.25)],
        [32, distribution(queue, chan=0, length=8, notes=musx.scale(72, 7, 2, 2, 1, 2, 2, 2, 1), dur=0.125)],
        [40, distribution(queue, chan=0, length=28, notes=musx.scale(48, 7, 2, 2, 1, 2, 2, 2, 1), dur=0.25)],
        [40, cc_linear(queue, chan=0, ctrl=32, length=16, start=0, end=64, low=0, high=127)],
        [56, distribution(queue, chan=0, length=16, notes=musx.scale(36, 7, 2, 2, 1, 2, 2, 2, 1), dur=0.5)],
        [60, distribution(queue, chan=0, length=28, notes=musx.scale(36, 7, 2, 2, 1, 2, 2, 2, 1), dur=0.25)],
        [64, distribution(queue, chan=0, length=24, notes=musx.scale(48, 7, 2, 2, 1, 2, 2, 2, 1), dur=0.125)],
        [68, distribution(queue, chan=0, length=20, notes=musx.scale(60, 7, 2, 2, 1, 2, 2, 2, 1), dur=0.125)],
        ])
    yield 0

In [41]:
def blown_bottles(queue):
    for iteration in range(24):
        jumble_gen = musx.jumble(musx.scale(48, 23, 2, 2, 1, 2, 2, 2, 1))
        notes = [next(jumble_gen) for _ in range(6)]
        positions = round(musx.rescale(musx.highran(), 0, 1, 4, 16))
        for note in notes:
            events = round(musx.rescale(musx.midran(), 0, 1, 1, positions))
            rotation = round(musx.rescale(musx.uniran(), 0, 1, 0, positions - 1))
            queue.compose([[iteration * 4, euclidean_rhythm(queue, chan=2, length=4, keynum=note, events=events, positions=positions, cycletime=4, rotation=rotation)]])
    yield 0

In [42]:
def swell(queue):
    queue.compose([
        [0, cc_linear(queue, chan=4, ctrl=35, length=12, start=31, end=56, low=0, high=127)],
        [0, random_ascend(queue, chan=4, length=60, notes=musx.scale(84, 17, 2, 2, 1, 2, 2, 2, 1), dur=0.5)],
        [0, cc_distribution(queue, chan=4, ctrl=33, length=60, rate=0.25)],
        [0, random_ascend(queue, chan=5, length=60, notes=musx.scale(72, 17, 2, 2, 1, 2, 2, 2, 1), dur=1)],
        [0, cc_distribution(queue, chan=4, ctrl=34, length=60, rate=0.5)],
        [30, cc_linear(queue, chan=4, ctrl=32, length=30, start=0, end=0.25)]
    ])
    yield 0

In [89]:
seq = musx.MidiSeq()
queue = musx.Scheduler(out=seq)
queue.compose([
    [0, cc_init(queue)],
    [0, ukulele(queue)],
    [88, drunk_walker(queue, chan=0, length=60, notes=musx.scale(36, 15, 2, 2, 1, 2, 2, 2, 1), dur=0.5)],
    [88, cc_linear(queue, chan=0, ctrl=33, length=60, start=64, end=48, low=0, high=127)],
    [98, blown_bottles(queue)],
    [98, cc_linear(queue, chan=2, ctrl=33, length=12, start=32, end=64, low=0, high=127)],
    [110, cc_linear(queue, chan=2, ctrl=32, length=60, start=0, end=1)],
    [140, swell(queue)],
])

Done!


In [90]:
seq.play(midiout, False)

In [69]:
seq = musx.MidiSeq()
queue = musx.Scheduler(out=seq)
queue.compose([
    [0, cc_init(queue)],
    # [0.01, cc_message(queue, chan=0, ctrl=32, value=63)],
    [0, drunk_walker(queue, chan=0, length=20, notes=musx.scale(36, 15, 2, 2, 1, 2, 2, 2, 1), dur=0.5)]
])
seq.play(midiout, False)

Done!
